In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from random import randint
import random
import numpy as np
import pandas as pd 
import torch.optim as optim
import os
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset
import torch
from scipy.stats import vonmises
import numpy.fft as fft 
import math
res = 12

In [ ]:
##Define the network to train on the data 
class CircleNet(nn.Sequential): 
    def __init__(self, res, dropout, n_out): 
        super(CircleNet, self).__init__()
        self.n_in = res
        self.affine1 = nn.Linear(self.n_in, 6)
        self.drop1 = nn.Dropout(dropout)
        ##self.affine2 = nn.Linear(8, 4)
        ##self.drop2 = nn.Dropout(dropout)
        self.affine3 = nn.Linear(6, n_out)
    def forward(self, x): 
        x = F.relu(self.affine1(x))
        x = self.drop1(x)
        ##x = F.relu(self.affine2(x))
        ##x = self.drop2(x)
        x = F.relu(self.affine3(x))
        x = F.softmax(x, dim = 1)
        return x

This is the Data Generation Code


In [ ]:
## make some uniform circular data
std = 2.76
mean = 8.5
res = 12
num_points = 100
num_samples = 100
data = np.zeros([num_points,res + 1])
for i in range(num_points): 
    for j in range(num_samples): 
        num = randint(0,res-1)
        data[i][num]+= 1
        data[i][res] = -1
    data[i][:-1] = (data[i][:-1]-mean)/std
np.savetxt("uniform12ptdata100test.csv", data, delimiter = ",")

In [ ]:
##Make Data for variable number of samples
num_points = 2500
res = 12
data = np.zeros([num_points, res+1])

std_factor = math.sqrt(1/res*(1-1/res))
for i in range(num_points):
    num_samples = randint(10,110)
    mean = num_samples/res
    for j in range(num_samples): 
        num = randint(0,res - 1)
        data[i][num] += 1
    data[i][res] = -1
    data[i][:-1] = (data[i][:-1] - mean)/std_factor/math.sqrt(num_samples)
np.savetxt("uniformvariablenumtest.csv", data, delimiter = ",")

In [ ]:
##Make Von Mises for variable number of samples
num_points = 2500
data = np.zeros([num_points,res+1])
std_factor = math.sqrt(1/res*(1-1/res))

for i in range(num_points):
    SAMPLE_SIZE = randint(10,110)
    kappa = 1
    rotate_amt = random.random()*2*np.pi
    rotate_amt2 = random.random()*2*np.pi
    mean = SAMPLE_SIZE/res
    vm_dist = vonmises.rvs(kappa, loc = rotate_amt, size=SAMPLE_SIZE)
  
    for l in range(SAMPLE_SIZE):
        vm_dist[l] = ((vm_dist[l] + np.pi)/(2*np.pi))%1*(res*10)//10
        
        data[i][int(vm_dist[l])] += 1
       
    ##rotate_amt = randint(0,res - 1)
    ##data[i,0:-1] = rotate(data[i,0:-1], rotate_amt)
    data[i][:-1] = (data[i][:-1] - mean)/std_factor/math.sqrt(SAMPLE_SIZE)
    data[i,-1] = -7

np.savetxt("vonmises12variabletest.csv", data, delimiter = ",")    

In [ ]:
np.std(data[10][:-1])

In [ ]:
data = np.zeros([10000,res+1])
for i in range(10000):
    SAMPLE_SIZE = 100
    kappa = 1
    rotate_amt = random.random()*2*np.pi
    vm_dist = vonmises.rvs(kappa, loc = rotate_amt, size=SAMPLE_SIZE)
    for l in range(SAMPLE_SIZE):
        random_rotate_factor = randint(0,1)
        vm_dist[l] = ((vm_dist[l] + np.pi + np.pi*random_rotate_factor)/(2*np.pi))%1*(res*10)//10
    
    for k in range(SAMPLE_SIZE):
        data[i][int(vm_dist[k])] += 1
    ##rotate_amt = randint(0,res - 1)
    ##data[i,0:-1] = rotate(data[i,0:-1], rotate_amt)
    data[i][:-1] = (data[i][:-1] - 12)/5
    data[i,-1] = -6
np.savetxt("bimodal8pt100.csv", data, delimiter = ",")    

In [ ]:
data = np.zeros([10000,res+1])
std = 3.7
mean = 8.5
for i in range(10000):
    SAMPLE_SIZE = 100
    kappa = 1.5
    rotate_amt = random.random()*2*np.pi
    rotate_amt2 = random.random()*2*np.pi
    vm_dist = vonmises.rvs(kappa, loc = rotate_amt, size=SAMPLE_SIZE//2)
    vm_dist2 = vonmises.rvs(kappa, loc = rotate_amt2, size = SAMPLE_SIZE//2)
    for l in range(SAMPLE_SIZE//2):
        vm_dist[l] = ((vm_dist[l] + np.pi)/(2*np.pi))%1*(res*10)//10
        vm_dist2[l] = ((vm_dist2[l] + np.pi)/(2*np.pi))%1*(res*10)//10
        data[i][int(vm_dist[l])] += 1
        data[i][int(vm_dist2[l])]+=1
    ##rotate_amt = randint(0,res - 1)
    ##data[i,0:-1] = rotate(data[i,0:-1], rotate_amt)
    data[i][:-1] = (data[i][:-1] - mean)/std
    data[i,-1] = -7
np.savetxt("vonmises12pt100.csv", data, delimiter = ",")    

In [ ]:
data = np.zeros([100,res+1])
for i in range(100):
    SAMPLE_SIZE = 100
    kappa = 1.5
    rotate_amt = random.random()*2*np.pi
    vm_dist = vonmises.rvs(kappa, loc = rotate_amt, size=SAMPLE_SIZE)
   
      
    vm_dist = ((vm_dist + np.pi)/(2*np.pi))%1*(res*10)//10
    
    for k in range(SAMPLE_SIZE):
        data[i][int(vm_dist[k])] += 1
    ##rotate_amt = randint(0,res - 1)
    ##data[i,0:-1] = rotate(data[i,0:-1], rotate_amt)
    data[i][:-1] = (data[i][:-1] - 12)/5
    data[i,-1] = -3
np.savetxt("vonmises8ptnormalizedtest.csv", data, delimiter = ",")    

In [ ]:
print(data[7])

In [ ]:
##This is a class that allows us to load data from csv files and make them into datasets that the dataloader can use
class CircularDataset(Dataset): 
    def __init__(self, data_root): 
        self.data = []
        self.dist_coder = LabelEncoder()
        self.dist_list = []
        for path in data_root: 
            array = pd.read_csv(path)
            
            self.dist_list += [array.values[0,-1]] ##get the distribution type for each file and add it to the list
            if len(self.data) == 0: 
                self.data = array.values
            else:
                print(self.data.shape)
                print(array.values.shape)
                self.data = np.concatenate((self.data,array.values), axis = 0)
        self.dist_coder.fit(self.dist_list)
    def __getitem__(self,idx): 
        return self.data[idx][0:-1], self.to_one_hot(self.dist_coder,([self.data[idx,-1]]))[0]
    def __len__(self): 
        return len(self.data)
    def to_one_hot(self, codec, values):
        value_idxs = codec.transform(values)
        return torch.eye(len(codec.classes_))[value_idxs]

This is the code where we train the network on the data

In [ ]:
def init_weights(m):
    if type(m) == nn.Linear:
        torch.nn.init.xavier_uniform_(m.weight)
        m.bias.data.fill_(0.05)
rate = .005
batchsize = 100
dataset = CircularDataset(["uniformvariablenum.csv","vonmises12variable.csv"])
testdata = CircularDataset(["uniformvariablenumtest.csv", "vonmises12variabletest.csv"])

iterator = torch.utils.data.DataLoader(dataset, batch_size = batchsize, shuffle = True)
testiterator = torch.utils.data.DataLoader(testdata, batch_size = 1, shuffle = True)
net = CircleNet(12,0,2)
optimizer = optim.Adam(net.parameters(),lr = rate)
running_loss = 0
netlist = []
epochs = 12
net.apply(init_weights)
net.train()
loss_func = nn.CrossEntropyLoss()
for e in range(epochs): 
    print(running_loss)
    running_loss = 0
    accuracy = 0
    i = 0
    for dist, labels in iterator: 
        
        
        optimizer.zero_grad()
        out = net(dist.float())
        
        loss = loss_func(out,torch.max(labels, 1)[1])
       
           
        loss.backward()
        optimizer.step()
        running_loss += loss
    for dist, labels in testiterator: 
        ps = net(dist.float())

        i += 1
        top_p, top_class = ps.topk(1, dim=1)
       
        top_p2, top_class2 = labels.topk(1, dim = 1)
        
        equals = top_class == top_class2
        
        accuracy += torch.sum(equals.float())
        
    
    print(accuracy)
    print(i)
    accuracy = 0
    

In [ ]:
dataset = CircularDataset(["uniformvariablenum.csv", "vonmises12variable.csv"])
def sum(x):
    tot = 0
    for i in x: 
        tot += i
    return tot

for i in range(100):
    print(sum(np.abs(dataset.__getitem__(i+25000)[0])))

In [ ]:
testdata = CircularDataset(["uniform12ptdata100.csv"])
testiterator = torch.utils.data.DataLoader(testdata, batch_size = 1, shuffle = True)
accuracy = 0

i = 0
for dist, labels in testiterator: 
    ps = net(dist.float())
        
    print(dist)
    i+=1
    top_p, top_class = ps.topk(1, dim=1)
    
    top_p2, top_class2 = labels.topk(1, dim = 1)
        
    equals = top_class == top_class2
   
    accuracy += torch.sum(equals.float())
        
print(accuracy/99)
print(i)


In [ ]:
def rotate(vec, num_rotations): 
    vec2 = np.zeros(len(vec))
    for i in range(len(vec)): 
        vec2[(i + num_rotations )% len(vec)] = vec[i]
    return vec2
data = np.zeros([100,res+1])
for i in range(100):
    SAMPLE_SIZE = 100
    kappa = 1
    vm_dist = vonmises.rvs(kappa, size=SAMPLE_SIZE)
    vm_dist = (vm_dist + np.pi)/(2*np.pi)*(res*10)//10
    
    for k in range(SAMPLE_SIZE):
        data[i][int(vm_dist[k])] += 1
    
    rotate_amt = randint(0,res - 1)
    data[i,0:-1] = rotate(data[i,0:-1], rotate_amt)
    data[i,-1] = -3
np.savetxt("vonmises8ptdata100test.csv", data, delimiter = ",")    

In [ ]:
paramlist2 = []
for parameter in net.parameters(): 
    paramlist2 += [parameter]

In [ ]:
savednet2 = net

In [ ]:
array2 = paramlist2[0].detach().numpy()
array2

In [ ]:
np.around(np.matmul(array2,fft.ifft(np.identity(12),12)),1)

In [ ]:
fft.irfft(np.identity(8),8)

In [ ]:
paramlist2

In [ ]:
net(torch.from_numpy(dataset.__getitem__(1)[0]).float()/20)

In [ ]:
torch.save(savednet2.state_dict(), "8x6x2bianduniexample.pt")

In [ ]:
for i, j in dataset:
    print(sum(i))

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import copy
mean = [0, 0]
cov = [[2/3, 0], [0, 2/3]]
x, y = np.random.multivariate_normal(mean, cov, 5000).T
x, y = abs(x), abs(y)

# Gen the data
a = np.random.multivariate_normal(mean, cov, 5000)
plt.plot([x[0] for x in a], [x[1] for x in a], 'x')
plt.axis('equal')
plt.show()

a = [np.absolute(x) for x in a]
plt.plot([x[0] for x in a], [x[1] for x in a], 'x')
plt.axis('equal')
plt.show()

a = np.asarray(a)
# project each value onto its maximum axis
row_maxes = a.max(axis=1).reshape(-1, 1)
a = np.where(a == row_maxes, row_maxes, 0)

plt.plot([x[0] for x in a], [x[1] for x in a], 'x')
plt.axis('equal')
plt.show()
    
def multidimsample(n, samples=1000):
    mean = [0 for i in range(n)]
    cov = [[0 if row == col else 1 for col in range(n)] for row in range(n)]
    res = np.random.multivariate_normal(mean, cov, samples)
    
    